In [11]:
import argparse
from dataclasses import dataclass
from pathlib import Path
from typing import Literal
import yaml
import graphviz

from ortools.constraint_solver import routing_enums_pb2, routing_parameters_pb2
from ortools.constraint_solver import pywrapcp

In [3]:
@dataclass(frozen=True)
class DataModel:
    weights: list
    demands: list
    vehicle_capacities: list
    depot: int
    
    @property
    def num_locations(self) -> int:
        return len(self.locations)
    
    @property
    def num_vehicles(self) -> int:
        return len(self.vehicle_capacities)

def _create_data_model(path: str) -> DataModel:
    with open(path, encoding="utf-8") as file:
        data = yaml.safe_load(file) 
    return DataModel(**data)

def _set_edge_weights(
    routing: pywrapcp.RoutingModel,
    manager: pywrapcp.RoutingIndexManager,
    data: DataModel) -> None:
    def _weight_callback(from_index: int, to_index: int) -> int:
        from_node: int = manager.IndexToNode(from_index)
        to_node:mint = manager.IndexToNode(to_index)
        return data.weights[from_node][to_node]

    weight_callback_index: int = routing.RegisterTransitCallback(_weight_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(weight_callback_index)

def _add_capacity_constraints(
    routing: pywrapcp.RoutingModel,
    manager: pywrapcp.RoutingIndexManager,
    data: DataModel) -> None:

    def _demand_callback(from_index: int) -> int:
        from_node: int = manager.IndexToNode(from_index)
        return data.demands[from_node]
    
    demand_callback_index: int = routing.RegisterUnaryTransitCallback(_demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        slack_max = 0,
        vehicle_capacities = data.vehicle_capacities,
        fix_start_cumul_to_zero=True,
        name="Capacity"
    )

In [4]:
@dataclass(frozen=True)
class Node:
    index: int
    load: float

def _create_node(
    manager: pywrapcp.RoutingIndexManager,
    assignment: pywrapcp.Assignment,
    capacity_dimension: pywrapcp.RoutingDimension,
    index: int
) -> Node:
    node_index: int = manger.IndexToNode(index)
    cap_var: pywrapcp.IntVar = capacity_dimension.CumulVar(index)
    load: int = assignment.Value(cap_var)
    
    return Node(index, load)

In [14]:
def _render(
    graph: graphviz.Digraph,
    filename: str,
    engine: Literal["dot", "neato", "fdp", "sfdp", "circo", "twopi", "osage"]
) -> None:
    ext = Path(filename).suffix or ".png"
    graph.render(cleanup= True, format= ext[1:], outfile=filename, engine=engine)
    

In [15]:
def _draw_network_graph(filename: str, data: DataModel) -> None:
    def _node_lable(index: int) -> str:
        if index == 0:
            return f"Node {index}\n Depot"
        return (
            f"Node {index}\nDemand; {data.demands[index]}\n"
        )
    
    graph = graphviz.Digraph(name="network")
    
    num_locations = data.num_locations
    
    for i in range (num_locations):
        for j in range(i+1, num_locations):
            name_i, name_j = f"Node{i}", f"Node{j}"
            graph.node(name=name_i, lable=_node_lable(i))
            graph.node(name=name_j, lable=_node_label(j))
            graph.edge(name_i, name_j, label=str(data().weights[i][j]))
    _render(graph=graph, filename=filename, engine='dot')
    
    print(f"The network graph has been saved to {filename}.")
    

In [ ]:
def _draw_route_graph(
    filename: str,
    data: DataModel,
    routing: pywrapcp.RoutingModel,
    manager: pywrapcp.RoutingIndexManager,
    assignment: pywrapcp.Assignment
) -> None:

    def _node_lable(index: int) -> str:
        if index == 0:
            return f"Node {index}\n Depot"
        return (
            f"Node {index}\nDemand; {data.demands[index]}\n"
        )
    
    graph = graphviz.Digraph(name="route")
    
    for vehicle_id in range(data.num_vehicles):
        index: int = routing.Start(vehicle_id)
        while not routing.IsEnd(index):
            node_index: int = manger.IndexToNode(index)
            next_var: pywrapcp.IntVar = routing.NextVar(index)
            next_index: int = assignment.Value(next_var)
            next_node_index: int = manger.IndexToNode(next_node_index)
            weight = data.weights[node_index][next_node_index]
            
            name_current, name_next = f"Node{node_index}", f"Node{next_node_index}"
            graph.node(name=name_current, label=_node_label(node_index))
            graph.node(name=name_next, label=_node_label(next_node_index))
            graph.edge(name_current, name_next, label=str(weight))
            
            index = next_index
            
    _render(graph=graph, filename=filename, engine="sfdp")
    
    print(f"The route graph has been saved to {filename}.")